In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-classification-finetuning/sample_submission.csv
/kaggle/input/llm-classification-finetuning/train.csv
/kaggle/input/llm-classification-finetuning/test.csv


In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [3]:
# 讀取資料
train = pd.read_csv("/kaggle/input/llm-classification-finetuning/train.csv")
test = pd.read_csv("/kaggle/input/llm-classification-finetuning/test.csv")

In [4]:
# 建立輸入文本欄位（prompt + response）
train['input_a'] = train['prompt'] + " " + train['response_a']
train['input_b'] = train['prompt'] + " " + train['response_b']
test['input_a'] = test['prompt'] + " " + test['response_a']
test['input_b'] = test['prompt'] + " " + test['response_b']

In [5]:
# winner_model_a/b/tie → label
def infer_label(row):
    if row["winner_model_a"] == 1:
        return 0
    elif row["winner_model_b"] == 1:
        return 1
    else:
        return 2

train['label'] = train.apply(infer_label, axis=1)


In [6]:
# 合併 input_a 和 input_b 為一個輸入文本
X_train = train['input_a'] + " ||| " + train['input_b']
y_train = train['label']
X_test = test['input_a'] + " ||| " + test['input_b']


In [7]:
# TF-IDF + LogisticRegression
model = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression(max_iter=1000, multi_class='multinomial'))
])
model.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 LogisticRegression(max_iter=1000, multi_class='multinomial'))])

In [8]:
# 預測機率
probs = model.predict_proba(X_test)

In [9]:
# 輸出 submission.csv（這是 Kaggle 評分系統要看的）
submission = pd.DataFrame({
    'id': test['id'],
    'winner_model_a': probs[:, 0],
    'winner_model_b': probs[:, 1],
    'winner_model_tie': probs[:, 2]
})
submission.to_csv("submission.csv", index=False)